In [5]:
from pysat.solvers import MinisatGH
from pysat.formula import CNF
# če vrne napako je treba: !pip install python-sat 

In [6]:
def subdivizija(G):
    # naredi subdivizijo grafa, na vsako povezavo postavi eno vozlišče
    H = G.copy()
    H.subdivide_edges(G.edges(), 1)
    return H

In [7]:
def poisci_pakirno_stevilo_SAT(G, l):
    # preveri ali ima graf G dopustno l-barvanje, če ga najde vrne dvojico (G, slovar barv)
    # x_v,i -> True, če je vozlišče v barve i, False sicer
    
    D = G.distance_matrix()
    V = G.vertices()
    
    cnf = CNF() # tukaj bomo shranili izraz za konjunktivno normalno obliko
    var_i = 1  # števec s katerim preimenujemo spremenljivke
    x = {}  # slovar kjer shranimo (vozlišče, barva) : števec

    # preimenujemo spremenljivke, to je potrebno ker morejo biti za cnf "int-i"
    for v in V:
        for i in range(1, l+1):
            x[(v, i)] = int(var_i) 
            var_i += 1

    # x[(v, i)] vozlišče v je barve i, -x[(v, i)] je negacija prejšnjega
    for v in V:
        cnf.append([x[(v, i)] for i in range(1, l+1)])  # vsako vozlišče je vsaj ene barve
        for i in range(1, l+1):
            for j in range(i+1, l+1):
                cnf.append([-x[(v, i)], -x[(v, j)]])  # nobeno vozlišče ni pobarvano z dvema barvama
    
    for u in V:
        for v in V:
            if u != v:
                for i in range(1, l+1):
                    if D[u][v] <= i:
                        cnf.append([-x[(u, i)], -x[(v, i)]])  # d(u, v) <= i --> biti morata različne barve
    
    # rešimo
    solver = MinisatGH()
    solver.append_formula(cnf.clauses)
    
    if solver.solve():
        # zapišemo slovar z barvami vozlišč
        model = solver.get_model()
        barve = {}
        for (v, i), var_i in x.items():
            if var_i in model:
                barve[v] = i

        return (G, barve)

    return "Ni dopustnega barvanja"

In [4]:
def stohasticno_iskanje_SAT(n, st_iteracij):
    # izvede stohastično iskanje na kubičnih grafih reda n, če najde graf katerega subdivizija ima pakirno št 5 ali več ga doda v seznam
    # najdeni kot trojico (G, pakirno stevilo, slovar barv)

    # kubični grafi so lahko samo tisti, ki imajo sodo število vozlišč
    if n % 2 != 0:
        return 'Število vozlišč mora biti sodo'

    # seznam grafov, ki smo jih že preverili
    preverjeni_grafi = set()

    # seznam grafov s pakirnim št > 4 ki ga vrnemo
    najdeni = []

    for _ in range(1, st_iteracij + 1):
        
        # generiramo naključen kubični graf
        G = graphs.RandomRegular(3, n)

        # če je dvodelen ga preskočimo
        if G.is_bipartite():
            continue

        # če je izomorfen kateremu od grafov, ki smo jih že preverili ga preskočimo
        G_canonical = G.canonical_label().copy(immutable=True)
        if G_canonical in preverjeni_grafi:
            continue

        # ga doda med preverjene
        preverjeni_grafi.add(G_canonical)
            
        sub_G = subdivizija(G)
        if not sub_G.is_connected():
            continue
            
        # preverimo ali ima subdivizija dopustno barvanje
        pakirno_st = 0
        for i in range(6, 3, -1): # pogledamo samo za i=6,5,4 ker nas če je pakirno <5 ne zanima
            rezultat = poisci_pakirno_stevilo_SAT(sub_G, i)
            if rezultat != "Ni dopustnega barvanja":
                sub_G, barve_vozlisc = rezultat
                pakirno_st = i
        
        # če je pakirno število > 5 dodamo v seznam
        if pakirno_st >= 5:
            najdeni.append((G, pakirno_st, barve_vozlisc))

    return najdeni

In [ ]:
%%time
# poženemo stohastično iskanje na grafih od 14 vozlišč do 24

st_iteracij = 2000
for n in range(14, 26, 2):
    ime_dat = f"stoh_SAT_{n}_voz"
    najdeni = stohasticno_iskanje_SAT(n, st_iteracij)
    save(najdeni, ime_dat)
    print(f"KONČANO {n}, cudni: {c}") 